Задание 1<br>
Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:<br>

оценка 2 и ниже — низкий рейтинг;<br>
оценка 4 и ниже — средний рейтинг;<br>
оценка 4.5 и 5 — высокий рейтинг.<br>
Результат классификации запишите в столбец class.<br>

In [6]:
import pandas as pd

def save_results(final_table, filename='data/functions_and_data/result_table.csv'):
    if final_table is not None:
        final_table.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"\nРезультаты сохранены в файл: {filename}")
        print(f"Сохранено {len(final_table)} записей")


def create_final_table():
    movies = pd.read_csv('data/functions_and_data/movies.csv',
                        header=None, names=['movieId', 'title', 'genres'])
    ratings = pd.read_csv('data/functions_and_data/ratings.csv',
                         header=None, names=['userId', 'movieId', 'rating', 'timestamp'])

    ratings['rating'] = pd.to_numeric(ratings['rating'], errors='coerce')
    ratings = ratings.dropna(subset=['rating'])

    avg_ratings = ratings.groupby('movieId')['rating'].mean().reset_index()
    avg_ratings.columns = ['movieId', 'средний рейтинг']

    result = pd.merge(avg_ratings, movies[['movieId', 'title']], on='movieId')

    def classify(r):
        if r <= 2: return 'низкий рейтинг'
        elif r <= 4: return 'средний рейтинг'
        else: return 'высокий рейтинг'

    result['class'] = result['средний рейтинг'].apply(classify)

    final = result[['title', 'средний рейтинг', 'class']].copy()
    final.columns = ['название фильма', 'средний рейтинг', 'class']
    final['средний рейтинг'] = final['средний рейтинг'].round(2)
    final = final.sort_values('средний рейтинг', ascending=False).reset_index(drop=True)

    return final

if __name__ == "__main__":
   save_results(create_final_table())


Результаты сохранены в файл: data/functions_and_data/result_table.csv
Сохранено 9066 записей


Задание 2<br>
Используйте файл keywords.csv.<br>

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность<br> определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’<br> пишется название этого региона. Если поисковый запрос не содержит названия<br> города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:<br>
<code/>
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}</code><br>

Результат классификации запишите в отдельный столбец region.<br>

In [1]:
import pandas as pd

def process_keyword_csv():
    df = pd.read_csv('data/functions_and_data/keywords.csv')
    if 'keyword' not in df.columns:
        df = pd.read_csv('data/functions_and_data/keywords.csv', header=None, names=['keyword', 'shows'])

    geo = {'Центр': ['москва', 'тула', 'ярославль'],
           'Северо-Запад': ['петербург', 'псков', 'мурманск'],
           'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

    def get_region(q):
        q = str(q).lower()
        return next((r for r, cities in geo.items() if any(c in q for c in cities)), 'undefined')

    df['region'] = df['keyword'].apply(get_region)

    df.to_csv('data/functions_and_data/keyword_result.csv', index=False)

    print(f"Результат сохранен в файл 'data/functions_and_data/keyword_result.csv'")
    print(f"Обработано записей: {len(df)}")

if __name__ == "__main__":
   process_keyword_csv()

Результат сохранен в файл 'data/functions_and_data/keyword_result.csv'
Обработано записей: 100000
